<a href="https://colab.research.google.com/github/Marvellous111/Alter/blob/main/Arrangohackathon_Alter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation requirements

In [ ]:
pip install python-arango --upgrade

In [ ]:
pip install arango-datasets

In [ ]:
pip install python-dotenv

In [13]:
pip install nx-arangodb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 71.5 MB/s eta 0:00:00
  Attempting uninstall: networkx
    Found existing installation: networkx 3.4.2
    Uninstalling networkx-3.4.2:
      Successfully uninstalled networkx-3.4.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; p

In [ ]:
!nvidia-smi
!nvcc --version

Sat Mar  8 22:03:37 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   38C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [11]:
pip install nx-cugraph-cu12 --extra-index-url https://pypi.nvidia.com

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com


In [12]:
pip install --upgrade langchain langchain-community langchain-openai langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 91.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.5/131.5 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.4/415.4 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 71.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.3 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.40
    Uninstalling langchain-core-0.3.40:
      Successfully uninstalled langchain-core-0.3.40
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.19
    Uninstalling langchain-0.3.19:
      Successfully uninstalled langchain-0.3.19


In [45]:
pip install -U langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 5.5 MB/s eta 0:00:00


# Import Statements

In [46]:
from arango import ArangoClient
from google.colab import userdata
from arango_datasets import Datasets

import base64
import networkx as nx
import nx_arangodb as nxadb

import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from random import randint
import re

from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI # We may likely never use this
from langchain_groq import ChatGroq
from langchain_community.graphs import ArangoGraph
from langchain_community.chains.graph_qa.arangodb import ArangoGraphQAChain
from langchain_core.tools import tool

# IMPORTANT NOTE
Google colab was used to build this notebook, hence the calls for api keys will be different from the usual calls, we are making use of the native ways to store secret keys using google colab.

If you were storing keys normally then consider using os.

# Client DB Section

In [16]:
db_username = userdata.get('ARANGO_DATABASE_USERNAME')
db_password = userdata.get('ARANGO_DATABASE_PASSWORD')
db_host_url = userdata.get('ARANGO_DATABASE_URL')
encoded_ca = userdata.get('ENCODED_CA')

try:
  file_content = base64.b64decode(encoded_ca)
  with open('cert_file.crt', 'w+') as f:
    f.write(file_content.decode("utf-8"))
except Exception as e:
  print(str(e))
  exit(1)

db = ArangoClient(hosts = db_host_url, verify_override="cert_file.crt").db(
  "_system",
  username=db_username,
  password=db_password,
  verify=True
  )

# Connect to datasets
datasets = Datasets(db)

# List datasets
print(datasets.list_datasets())

# List more information about a particular dataset
print(datasets.dataset_info("OPEN_INTELLIGENCE"))

# Load a dataset
open_intelligence_dataset = datasets.load("OPEN_INTELLIGENCE")




Output()

['IMDB_X', 'IMDB_PLATFORM', 'AMAZON_COMPUTER_PRODUCTS', 'AMAZON_PRODUCT_WITH_EMBEDDINGS', 'FAKE_HOMO_MULTICLASS', 'FAKE_HOMO_BINARY', 'FAKE_HETRO_LARGE_MULTICLASS', 'MAG', 'OPEN_INTELLIGENCE', 'OPEN_INTELLIGENCE_ANGOLA', 'FLIGHTS', 'CORA', 'DBLP', 'CVE', 'MTCARS', 'P2PSB', 'SCHAIN', 'ITSM_SAGE', 'ABIDE', 'CYBER', 'GAME_OF_THRONES', 'MARKET', 'SYNTHEA_P1000', 'SYNTHEA_P100']
{'files': 14, 'edges': [{'collection_name': 'eventActor', 'files': ['https://arangodb-dataset-library-ml.s3.amazonaws.com/open_intelligence/edge_collection/eventActor.json']}, {'collection_name': 'hasLocation', 'files': ['https://arangodb-dataset-library-ml.s3.amazonaws.com/open_intelligence/edge_collection/hasLocation.json']}, {'collection_name': 'hasSource', 'files': ['https://arangodb-dataset-library-ml.s3.amazonaws.com/open_intelligence/edge_collection/hasSource.json']}, {'collection_name': 'inCountry', 'files': ['https://arangodb-dataset-library-ml.s3.amazonaws.com/open_intelligence/edge_collection/inCountry.js

Output()

Output()

Initializing collection 'Class'


...

Output()

Output()

Initializing collection 'type'


Output()

Output()

In [21]:
'''
This cell is most likely not needed since the cell above this already does the hard work for us.
NOTE: If you decide to use the pre-loaded dataset, then this cell is not needed.
'''
G_adb = nxadb.Graph(
    name="OPEN_INTELLIGENCE",
    db=db,
    incoming_graph_data=datasets.load("OPEN_INTELLIGENCE"),
    write_batch_size=50000 # feel free to modify
)

print(G_adb)

[22:37:57 +0000] [INFO]: Graph 'Open_Intelligence' created.
INFO:nx_arangodb:Graph 'Open_Intelligence' created.


Graph named 'Open_Intelligence' with 0 nodes and 0 edges


In [26]:
G_adb = nxadb.MultiDiGraph(name="OPEN_INTELLIGENCE", db=db)
print(G_adb)

[22:54:07 +0000] [INFO]: Graph 'OPEN_INTELLIGENCE' exists.
INFO:nx_arangodb:Graph 'OPEN_INTELLIGENCE' exists.
[22:54:08 +0000] [INFO]: Default node type set to 'Actor'
INFO:nx_arangodb:Default node type set to 'Actor'


MultiDiGraph named 'OPEN_INTELLIGENCE' with 80047 nodes and 321819 edges


In [31]:
'''
Lets sample 3 nodes and 3 edges here to test the graph
'''
#Sample 3 nodes
result = G_adb.query("""
    FOR node IN Actor
        SORT RAND()
        LIMIT 3
        RETURN node
""")
print(list(result))
print('-'*10)

# Sample 3 edges
result = G_adb.query("""
    FOR edge IN Country
        SORT RAND()
        LIMIT 3
        RETURN edge
""")
print(list(result))
print('-'*10)

# Traverse a node's 1-hop neighborhood
result = G_adb.query("""
    FOR node, edge, path IN 1..1 ANY 'Actor/1' GRAPH OPEN_INTELLIGENCE
        LIMIT 1
        RETURN path
""")

print(list(result))

[{'_key': 'ac052437b096366d7212ba99c69555fc', '_id': 'Actor/ac052437b096366d7212ba99c69555fc', '_rev': '_jVdhj_a-_b', 'name': 'Kouyeya Communal Militia (Guinea)'}, {'_key': '7d32a6799b45729e0b96d3885e3b1b7c', '_id': 'Actor/7d32a6799b45729e0b96d3885e3b1b7c', '_rev': '_jVdhi2O--T', 'name': 'Militia (Moise Tshiombe)'}, {'_key': '31e9865fe675e164e1d22b5bea208b2a', '_id': 'Actor/31e9865fe675e164e1d22b5bea208b2a', '_rev': '_jVdhi2G--7', 'name': 'Niagassola Communal Militia (Guinea)'}]
----------
[{'_key': 'Botswana', '_id': 'Country/Botswana', '_rev': '_jVdhkcy--A', 'name': 'Botswana'}, {'_key': 'Equatorial_Guinea', '_id': 'Country/Equatorial_Guinea', '_rev': '_jVdhkcy--I', 'name': 'Equatorial Guinea'}, {'_key': 'Republic_of_Congo', '_id': 'Country/Republic_of_Congo', '_rev': '_jVdhkcy--F', 'name': 'Republic of Congo'}]
----------
[]


# App build using langchain and LangGraph

In [54]:
# This is the langchain wrapper for the app

arango_graph = ArangoGraph(db)

# LANGUAGE MODEL
Groq was used for the api with llama-70B for the model to be used due to its low and basically free cost of use. It should also work well with OpenA langchain so there shouldn't be an issue, if there is an issue then you can switch to OpenAI chat or use the deepseek chat. But note that you must have credits for those two

In [49]:
deepseek_api_key = userdata.get('DEEPSEEK_API_KEY')
groq_api_key = userdata.get('API_KEY_GROQ')
#We have the OpenAI key just incase we want to use it as well
open_api_key = userdata.get('OPENAI_API_KEY')

In [52]:
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0.2,
    max_retries=2,
    api_key=groq_api_key,
    verbose=True,
    )

In [53]:
messages = [
    ("system", "You are a helpful translator. Translate the user sentence to French."),
    ("human", "I love programming in rust and python."),
]
llm.invoke(messages)

AIMessage(content="J'adore programmer en Rust et en Python.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 56, 'total_tokens': 68, 'completion_time': 0.043636364, 'prompt_time': 0.00515431, 'queue_time': 0.228885884, 'total_time': 0.048790674}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_5d5eebccf4', 'finish_reason': 'stop', 'logprobs': None}, id='run-4af13869-9aaa-47c9-91aa-09031e04e1ea-0', usage_metadata={'input_tokens': 56, 'output_tokens': 12, 'total_tokens': 68})

# TOOLS FOR LANG GRAPH
These are the tools to use for the app.

In [55]:
@tool
def hybrid_text_to_aql_and_network_to_text(query: str) -> str:
    """
    This hybrid tool processes a natural language query by:
      1. Translating it into an AQL query and executing it on the ArangoDB graph.
      2. Determining if additional network analytics are needed (e.g., for queries like
         "Who are Alice's most influential connections?").
      3. If so, passing the subgraph to a NetworkX/cuGraph analysis function (e.g., PageRank).
      4. Synthesizing a final natural language response combining the direct AQL result
         and the analytic insights.

    Parameters:
        query (str): The user's natural language query.

    Returns:
        str: The final natural language response.
    """
    try:
        # Initialize the LLM with deterministic settings for clarity.
        llm = ChatOpenAI(
          model="llama-3.3-70b-versatile",
          temperature=0.2,
          max_retries=2,
          api_key=groq_api_key,
          verbose=True,
        )

        # Step 1: Execute the AQL conversion and query using LangChain's integration.
        aql_chain = ArangoGraphQAChain.from_llm(
            llm=llm,
            graph=arango_graph,  # 'arango_graph' should be defined/initialized elsewhere.
            verbose=True,
            allow_dangerous_requests=True
        )
        aql_result = aql_chain.invoke(query)
        aql_text = str(aql_result["result"])

        # Step 2: Determine if network analytics are needed.
        # (For example, if the query contains terms like "influential", "central", "impact", etc.)
        trigger_keywords = ["influential","central","network","relationship","connection","alliance","conflict","protest","unrest","terror","sanction","diplomacy","government","policy","actor","coalition","neighbor","mapping","geo","location","city","regional","power","centrality","influence","engagement","affiliation","tension","escalation","stability"]
        if any(keyword in query.lower() for keyword in trigger_keywords):
            # Retrieve a subgraph via AQL traversal (you may customize this retrieval as needed)
            subgraph_data = arango_graph.traverse(start_node="United States", depth=2)  # Example: traversal from "United states"

            # Run GPU-accelerated analytics (e.g., PageRank using cuGraph)
            # Assume `run_network_analytics` is a custom function that accepts subgraph data and returns analysis insights.
            analytics_result = run_network_analytics(subgraph_data, method="pagerank")

            combined_info = f"{aql_text}\n\nNetwork Analytics Insights:\n{analytics_result}"
        else:
            combined_info = aql_text

        # Step 3: Synthesize the final natural language response.
        synthesis_prompt = (
            f"Based on the following information from an AQL query and network analysis, "
            f"provide a clear and concise answer:\n\n{combined_info}"
        )
        synthesis_response = llm(synthesis_prompt)

        return synthesis_response.strip()

    except Exception as e:
        return f"Error executing hybrid query: {e}"


def run_network_analytics(subgraph_data, method="pagerank") -> str:
    """
    Runs network analytics on the provided subgraph data.

    Parameters:
        subgraph_data: A subgraph retrieved from ArangoDB (expected to be a NetworkX-compatible graph).
        method (str): The network analytic method to use (we are using pagerank).

    Returns:
        str: We want to return a summary of analytic insights (e.g., top nodes by PageRank).
    """

    try:
        # Assume subgraph_data is already a NetworkX graph.
        G = subgraph_data

        if method.lower() == "pagerank":
            pr = nx.pagerank(G, alpha=0.85)
            # Sort nodes by PageRank score in descending order.
            sorted_pr = sorted(pr.items(), key=lambda x: x[1], reverse=True)
            top_nodes = sorted_pr[:5]
            result_text = "Top nodes by PageRank:\n"
            for node, score in top_nodes:
                result_text += f"{node}: {score:.4f}\n"
            return result_text
        else:
            return "Selected network analytic method is not implemented."
    except Exception as e:
        return f"Error in network analytics: {e}"


In [56]:
import re
import networkx as nx

@tool
def text_to_nx_algorithm_to_text(query: str) -> str:
    """
    This tool accepts a natural language graph analysis query, generates Python code
    (using NetworkX) to analyze the ArangoDB-derived graph (G_adb), executes the code,
    and synthesizes a concise natural language answer.

    We want to use this tool only for queries requiring deep network analysis (e.g., shortest path,
    PageRank, centrality measures). If the query can be solved using AQL alone, then we abstain.

    Parameters:
        query (str): The natural language graph analysis query.

    Returns:
        str: We want to return a synthesized natural language answer based on the analysis.
    """
    try:
        # Initialize the LLM with deterministic settings for clarity.
        llm = ChatOpenAI(
          model="llama-3.3-70b-versatile",
          temperature=0.2,
          max_retries=2,
          api_key=groq_api_key,
          verbose=True,
        )

        # Step 1: Generate Python code for network analysis using NetworkX.
        prompt_code = f"""
I have a NetworkX Graph called `G_adb` representing a GDELT-derived graph.
The graph follows this schema: {arango_graph.schema}
I need to answer the following graph analysis query: {query}
Generate Python code that uses NetworkX to perform the appropriate analysis on `G_adb`.
Be very precise in selecting the correct algorithm (e.g., PageRank, shortest path, centrality measures).
Store the final answer in a variable called FINAL_RESULT.
Output only the Python code (no markdown or extra explanations).
"""
        code_response = llm.invoke(prompt_code).content

        # Clean the generated code (remove markdown formatting if present)
        generated_code = re.sub(r"^```(?:python)?\n|```$", "", code_response, flags=re.MULTILINE).strip()

        # Debug: Print the generated code for verification.
        print("Generated NetworkX code:")
        print("-" * 40)
        print(generated_code)
        print("-" * 40)

        # Step 2: Execute the generated code.
        # Ensure that the global namespace includes your NetworkX graph and the nx module.
        global_namespace = {"G_adb": G_adb, "nx": nx}
        local_namespace = {}

        try:
            exec(generated_code, global_namespace, local_namespace)
        except Exception as exec_error:
            print(f"Error executing generated code: {exec_error}")
            return f"Error executing generated code: {exec_error}"

        if "FINAL_RESULT" not in local_namespace:
            return "Error: FINAL_RESULT variable not found in the executed code."
        final_result = local_namespace["FINAL_RESULT"]

        # Step 3: Synthesize the final natural language answer using the Llama model.
        synthesis_prompt = f"""
I have a NetworkX Graph called `G_adb` with the following schema: {arango_graph.schema}
The following Python code was generated and executed to analyze the graph for this query:
{query}
The result of the analysis is stored in FINAL_RESULT, which is: {final_result}.
Based on the original query and this result, generate a short, clear, and concise answer.
Your answer should directly address the query.
"""
        final_answer = llm.invoke(synthesis_prompt).content
        return final_answer.strip()

    except Exception as e:
        return f"Error in text_to_nx_algorithm_to_text tool: {e}"
